In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
file_path = r'C:\Users\rafael.s.fagundes\Downloads\_Clone__AEVDT_6129___VMOS_Dashboard_Test.csv'

df = pd.read_csv(file_path)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502217 entries, 0 to 502216
Data columns (total 19 columns):
 #   Column                                          Non-Null Count   Dtype 
---  ------                                          --------------   ----- 
 0   year                                            502217 non-null  int64 
 1   month                                           502217 non-null  int64 
 2   dim_market_id__market_name                      502217 non-null  object
 3   dim_device_category_id__device_category_name    502217 non-null  object
 4   dim_page_id                                     502217 non-null  int64 
 5   dim_page_id__page_name                          483958 non-null  object
 6   dim_page_template_id__page_template_key         502217 non-null  object
 7   dim_page_template_id__page_template_name        502217 non-null  object
 8   dim_event_id__event_name                        502217 non-null  object
 9   dim_event_action_id__event_action_nam

In [4]:
df.head()

,year,month,dim_market_id__market_name,dim_device_category_id__device_category_name,dim_page_id,dim_page_id__page_name,dim_page_template_id__page_template_key,dim_page_template_id__page_template_name,dim_event_id__event_name,dim_event_action_id__event_action_name,dim_event_category_id__event_category_name,page_path,dim_client_id_count,client_key_count,global__filter__header_and_footer_events_count,filter__non_interaction_events_count,interacting_users_count,users_model_overview_count,engagers_model_overview_count
0,2024,3,Brazil,Mobile,-2493890501690526571,models,generic-home-template,Generic Home Template,Filter,custom_sub_group,Filter,/passengercars/models.html,5609,5609,0,0,5609,5609,0
1,2023,5,Turkey,Mobile,3943889068410560298,passengercars,generic-home-template,Generic Home Template,unknown,model_overview,Impression,/passengercars.html,215989,215989,0,215989,215989,215989,0
2,2023,9,Germany,Desktop,-7444071767201028348,NaN,unknown,unknown,unknown,model_overview,Impression,/passengercars/models.html,13191,13191,0,13191,13191,13191,0
3,2024,9,Austria,Mobile,-2493890501690526571,models,generic-template,Generic Template,Filter,custom_group,Filter,/passengercars/models.html,2734,2734,0,0,2734,2734,0
4,2023,6,Taiwan,Mobile,3943889068410560298,passengercars,generic-home-template,Generic Home Template,unknown,owc-header|owc-header,Link,/passengercars.html,35031,35031,35031,0,35031,35031,35031


In [5]:
# top = df.groupby(by=['dim_application_id__application_key'])['client_key_count'].sum().sort_values(ascending=False).reset_index()
# top = top.head(10)

In [6]:
# df_top_filtered = df[
#     df['dim_application_id__application_key'].isin(top['dim_application_id__application_key'])
# ]

In [7]:
# df_top_filtered = df_top_filtered[df_top_filtered['dim_application_id__application_key'] == 'vmos']

In [8]:
groupby_cols = [
    'year',
    'month',
    'dim_market_id__market_name',
    # 'dim_application_id__application_key',
    # 'dim_device_category_id__device_category_name',
    'dim_page_id',
    'dim_page_id__page_name',
    'dim_page_template_id__page_template_key',
    'page_path',
    'dim_event_id__event_name',
    'dim_event_action_id__event_action_name',
    'dim_event_category_id__event_category_name'
]

agg_cols = [
    'client_key_count',
    'global__filter__header_and_footer_events_count',
    'filter__non_interaction_events_count',
    'interacting_users_count',
    'users_model_overview_count',
    'engagers_model_overview_count'
]

df_groupby = (
    df
    .groupby(by=groupby_cols)[agg_cols]
    .sum()
    .reset_index()
)


In [9]:
# Dictionary mapping 'cd2' values to corresponding regex patterns for page categorization
cd2dict = {
    'all_models_overview': ['^models$'],
    'car_configurator': [
        '^car-configurator\..+$', 
        '^configurator$'
    ],
    'dealer_locator': [
        '^dealer-locator$', 
        '^content-pool\.applications\.dealer-locator$'
    ],
    'homepage': [
        '^passengercars$', 
        '^vans$'
    ],
    'pi_pages': ['^models\..+$'],
    'shop': [
        '^new-cars-storefront$', 
        '^product_list\..*$', 
        '^product_list$', 
        '^product_detail$', 
        '^product_detail\..*$', 
        '^checkout$', 
        '^checkout_vehicles$', 
        '^checkout_step.*$', 
        '^cart_overview$', 
        '^buy\.new-car.*$', 
        '^business-cars-storefront$', 
        '^buy\.business-car$', 
        '^invoice_and_payment$', 
        '^order_confirmation$', 
        '^order_overview$', 
        '^home$', 
        '^home.*$', 
        '^search-results$', 
        '^category_overview$', 
        '^online-showroom$'
    ],
    'test_drive': [
        '^test-drive$', 
        '^online-testdrive$', 
        '^online-testdrive\..*$'
    ]
}


<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:10: SyntaxWarning: invalid escape sequence '\.'
<>:16: SyntaxWarning: invalid escape sequence '\.'
<>:19: SyntaxWarning: invalid escape sequence '\.'
<>:22: SyntaxWarning: invalid escape sequence '\.'
<>:27: SyntaxWarning: invalid escape sequence '\.'
<>:29: SyntaxWarning: invalid escape sequence '\.'
<>:42: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:10: SyntaxWarning: invalid escape sequence '\.'
<>:16: SyntaxWarning: invalid escape sequence '\.'
<>:19: SyntaxWarning: invalid escape sequence '\.'
<>:22: SyntaxWarning: invalid escape sequence '\.'
<>:27: SyntaxWarning: invalid escape sequence '\.'
<>:29: SyntaxWarning: invalid escape sequence '\.'
<>:42: SyntaxWarning: invalid escape sequence '\.'
C:\Users\rafael.s.fagundes\AppData\Local\Temp\ipykernel_12768\272535557.py:5: SyntaxWarning: invalid escape sequence '\.'
  '^car-configurator\..+$',
C:\Users\rafael.s.fagundes\AppData\L

In [10]:
# Função simples para encontrar a categoria de uma página
def get_category(page_name):
    for category, patterns in cd2dict.items():
        for pattern in patterns:
            if re.match(pattern, page_name):
                return category  # Retorna a primeira categoria encontrada
    return 'Unknown'  # Se não encontrar, retorna "Unknown"

In [11]:
def generate_page_users(df):
    """
    Adiciona a coluna `page_users` ao DataFrame Pandas com base nas categorias de páginas existentes.

    :param df: DataFrame Pandas contendo os dados de navegação dos usuários.
    :return: DataFrame atualizado com a coluna `page_users`.
    """

    # Definição das condições
    conditions = (
        # Homepage - Mercedes-Benz Passenger Cars
        (df['category'] == 'passengercars') |
        (df['dim_page_template_id__page_template_key'] == 'generic-home-template') |

        # Vans Homepage
        ((df['category'] == 'vans') & (df['dim_page_template_id__page_template_key'] == 'generic-home-template')) |

        # Model Overview (MO) Pages
        (df['category'] == 'models') &
        (~df['dim_page_template_id__page_template_key'].isin(['generic-product-template', 'productdefinition'])) |

        # Test Drive Page
        (df['category'] == 'test-drive') |

        # Configurator Page
        (df['category'] == 'configurator') |

        # Vans Model Overview
        (df['category'] == 'vans.models')
    )

    # Criando a nova coluna 'page_users'
    df['page_users'] = np.where(conditions, df['client_key_count'], np.nan)

    return df


In [12]:
# Aplicar a função à coluna 'page_name'
df_groupby['category'] = df_groupby['dim_page_id__page_name'].apply(get_category)

In [13]:
df_groupby.head()

,year,month,dim_market_id__market_name,dim_page_id,dim_page_id__page_name,dim_page_template_id__page_template_key,page_path,dim_event_id__event_name,dim_event_action_id__event_action_name,dim_event_category_id__event_category_name,client_key_count,global__filter__header_and_footer_events_count,filter__non_interaction_events_count,interacting_users_count,users_model_overview_count,engagers_model_overview_count,category
0,2023,1,Argentina,-7753536518500320893,configurator,vmoscs,/passengercars/configurator.html,unknown,custom_group,Filter,212,0,0,212,212,0,car_configurator
1,2023,1,Argentina,-7753536518500320893,configurator,vmoscs,/passengercars/configurator.html,unknown,custom_sub_group,Filter,196,0,0,196,196,0,car_configurator
2,2023,1,Argentina,-7753536518500320893,configurator,vmoscs,/passengercars/configurator.html,unknown,model_gallery,Feature,702,0,0,702,702,0,car_configurator
3,2023,1,Argentina,-7753536518500320893,configurator,vmoscs,/passengercars/configurator.html,unknown,model_gallery,Link,299,0,0,299,299,299,car_configurator
4,2023,1,Argentina,-7753536518500320893,configurator,vmoscs,/passengercars/configurator.html,unknown,model_menu,Link,140,0,0,140,140,140,car_configurator


In [14]:
df_final = generate_page_users(df_groupby)

In [15]:
df_final.head()

,year,month,dim_market_id__market_name,dim_page_id,dim_page_id__page_name,dim_page_template_id__page_template_key,page_path,dim_event_id__event_name,dim_event_action_id__event_action_name,dim_event_category_id__event_category_name,client_key_count,global__filter__header_and_footer_events_count,filter__non_interaction_events_count,interacting_users_count,users_model_overview_count,engagers_model_overview_count,category,page_users
0,2023,1,Argentina,-7753536518500320893,configurator,vmoscs,/passengercars/configurator.html,unknown,custom_group,Filter,212,0,0,212,212,0,car_configurator,NaN
1,2023,1,Argentina,-7753536518500320893,configurator,vmoscs,/passengercars/configurator.html,unknown,custom_sub_group,Filter,196,0,0,196,196,0,car_configurator,NaN
2,2023,1,Argentina,-7753536518500320893,configurator,vmoscs,/passengercars/configurator.html,unknown,model_gallery,Feature,702,0,0,702,702,0,car_configurator,NaN
3,2023,1,Argentina,-7753536518500320893,configurator,vmoscs,/passengercars/configurator.html,unknown,model_gallery,Link,299,0,0,299,299,299,car_configurator,NaN
4,2023,1,Argentina,-7753536518500320893,configurator,vmoscs,/passengercars/configurator.html,unknown,model_menu,Link,140,0,0,140,140,140,car_configurator,NaN


In [16]:
df_groupby.to_csv('test.csv')